# Experiment Analysis

Consolidated analysis notebook for distributive justice experiments.

**Key Features:**
- Load results from simplified experiment files
- Statistical analysis of principle preferences
- Visualization of consensus patterns
- Export results for further analysis

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Analysis environment initialized!")

## 1. Load Experiment Results

In [ ]:
def load_experiment_results(experiment_ids: List[str]) -> List[Dict[str, Any]]:
    """
    Load experiment results from JSON files.
    
    Args:
        experiment_ids: List of experiment IDs to load
    
    Returns:
        List of experiment result dictionaries
    """
    results = []
    results_dir = Path("experiment_results")
    
    for exp_id in experiment_ids:
        # Try different file naming patterns
        possible_files = [
            results_dir / f"{exp_id}.json",
            results_dir / f"{exp_id}_complete.json"
        ]
        
        for file_path in possible_files:
            if file_path.exists():
                try:
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                    results.append(data)
                    print(f"✅ Loaded: {file_path}")
                    break
                except Exception as e:
                    print(f"❌ Error loading {file_path}: {e}")
        else:
            print(f"⚠️  Not found: {exp_id}")
    
    return results

# Load recent experiment results
experiment_ids = ["real_test", "batch_test_001", "batch_test_002"]
experiments = load_experiment_results(experiment_ids)

print(f"\n📊 Loaded {len(experiments)} experiments")

## 2. Data Processing

In [ ]:
def extract_experiment_summary(experiment_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Extract key summary information from experiment data.
    """
    try:
        metadata = experiment_data.get('experiment_metadata', {})
        
        summary = {
            'experiment_id': metadata.get('experiment_id', 'unknown'),
            'num_agents': metadata.get('num_agents', 0),
            'max_rounds': metadata.get('max_rounds', 0),
            'consensus_reached': metadata.get('consensus_reached', False),
            'total_duration': metadata.get('total_duration_seconds', 0),
            'rounds_to_consensus': metadata.get('rounds_to_consensus', 0),
            'agreed_principle': metadata.get('agreed_principle', {}),
            'total_messages': len(experiment_data.get('deliberation_transcript', []))
        }
        
        # Extract agreed principle name
        if summary['agreed_principle']:
            summary['agreed_principle_name'] = summary['agreed_principle'].get('principle_name', 'Unknown')
            summary['agreed_principle_id'] = summary['agreed_principle'].get('principle_id', 0)
        else:
            summary['agreed_principle_name'] = None
            summary['agreed_principle_id'] = None
        
        return summary
        
    except Exception as e:
        print(f"Error extracting summary: {e}")
        return {}

# Process all experiments
if experiments:
    experiment_summaries = []
    
    for exp_data in experiments:
        summary = extract_experiment_summary(exp_data)
        if summary:
            experiment_summaries.append(summary)
    
    # Create DataFrame
    df = pd.DataFrame(experiment_summaries)
    
    print(f"📊 Processed {len(df)} experiments")
    print("\nExperiment Summary:")
    print(df[['experiment_id', 'consensus_reached', 'total_duration', 'agreed_principle_name']])
else:
    print("⚠️  No experiments loaded")
    df = pd.DataFrame()

## 3. Basic Statistics

In [ ]:
if not df.empty:
    print("📊 EXPERIMENT STATISTICS")
    print("=" * 50)
    
    # Basic stats
    print(f"Total experiments: {len(df)}")
    print(f"Consensus reached: {df['consensus_reached'].sum()} ({df['consensus_reached'].mean():.1%})")
    print(f"Average duration: {df['total_duration'].mean():.1f} seconds")
    print(f"Average rounds to consensus: {df['rounds_to_consensus'].mean():.1f}")
    print(f"Average messages: {df['total_messages'].mean():.1f}")
    
    # Principle preferences
    if df['agreed_principle_name'].notna().any():
        print("\n🎯 Principle Preferences:")
        principle_counts = df[df['consensus_reached']]['agreed_principle_name'].value_counts()
        for principle, count in principle_counts.items():
            print(f"   {principle}: {count} experiments")
    
    # Performance metrics
    print("\n⏱️  Performance Metrics:")
    print(f"   Fastest experiment: {df['total_duration'].min():.1f}s")
    print(f"   Slowest experiment: {df['total_duration'].max():.1f}s")
    print(f"   Median duration: {df['total_duration'].median():.1f}s")
    
else:
    print("⚠️  No data available for analysis")

## 4. Visualizations

In [ ]:
if not df.empty:
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Experiment Analysis Dashboard', fontsize=16)
    
    # 1. Consensus rate
    consensus_counts = df['consensus_reached'].value_counts()
    axes[0, 0].pie(consensus_counts.values, labels=['Consensus' if x else 'No Consensus' for x in consensus_counts.index], 
                   autopct='%1.1f%%', startangle=90)
    axes[0, 0].set_title('Consensus Rate')
    
    # 2. Duration distribution
    axes[0, 1].hist(df['total_duration'], bins=max(3, len(df)//2), alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Duration (seconds)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Duration Distribution')
    
    # 3. Principle preferences
    if df['agreed_principle_name'].notna().any():
        principle_counts = df[df['consensus_reached']]['agreed_principle_name'].value_counts()
        axes[1, 0].bar(range(len(principle_counts)), principle_counts.values)
        axes[1, 0].set_xticks(range(len(principle_counts)))
        axes[1, 0].set_xticklabels([name[:20] + '...' if len(name) > 20 else name for name in principle_counts.index], 
                                  rotation=45, ha='right')
        axes[1, 0].set_ylabel('Count')
        axes[1, 0].set_title('Agreed Principles')
    else:
        axes[1, 0].text(0.5, 0.5, 'No consensus data available', ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Agreed Principles')
    
    # 4. Rounds to consensus
    if df['rounds_to_consensus'].sum() > 0:
        consensus_df = df[df['consensus_reached']]
        if len(consensus_df) > 0:
            axes[1, 1].bar(consensus_df['experiment_id'], consensus_df['rounds_to_consensus'])
            axes[1, 1].set_xlabel('Experiment')
            axes[1, 1].set_ylabel('Rounds to Consensus')
            axes[1, 1].set_title('Rounds to Consensus by Experiment')
            axes[1, 1].tick_params(axis='x', rotation=45)
        else:
            axes[1, 1].text(0.5, 0.5, 'No consensus reached', ha='center', va='center', transform=axes[1, 1].transAxes)
            axes[1, 1].set_title('Rounds to Consensus')
    else:
        axes[1, 1].text(0.5, 0.5, 'No rounds data available', ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Rounds to Consensus')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️  No data available for visualization")

## 5. Export Results

In [ ]:
if not df.empty:
    # Export analysis results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export summary data
    export_file = f"analysis_results_{timestamp}.csv"
    df.to_csv(export_file, index=False)
    print(f"📊 Summary data exported to: {export_file}")
    
    # Export analysis summary
    analysis_summary = {
        'timestamp': timestamp,
        'total_experiments': len(df),
        'consensus_rate': df['consensus_reached'].mean(),
        'average_duration': df['total_duration'].mean(),
        'average_rounds': df['rounds_to_consensus'].mean(),
        'principle_preferences': df[df['consensus_reached']]['agreed_principle_name'].value_counts().to_dict() if df['agreed_principle_name'].notna().any() else {}
    }
    
    summary_file = f"analysis_summary_{timestamp}.json"
    with open(summary_file, 'w') as f:
        json.dump(analysis_summary, f, indent=2)
    
    print(f"📊 Analysis summary exported to: {summary_file}")
    
    # Display final summary
    print("\n🎯 FINAL ANALYSIS SUMMARY")
    print("=" * 50)
    print(f"Experiments analyzed: {len(df)}")
    print(f"Consensus rate: {df['consensus_reached'].mean():.1%}")
    print(f"Average duration: {df['total_duration'].mean():.1f} seconds")
    print(f"Most common principle: {df[df['consensus_reached']]['agreed_principle_name'].mode().iloc[0] if df['consensus_reached'].any() and df['agreed_principle_name'].notna().any() else 'None'}")
    print(f"Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
else:
    print("⚠️  No data to export")

## 6. Run New Experiments

Use the simplified experiment system to run new experiments:

In [ ]:
# Example: Generate and run new experiments
from config_generator import create_test_generator
from run_experiment import run_experiment_sync
from run_batch import run_batch_sync

# Create a new test configuration
print("🔧 Creating new test configuration...")
generator = create_test_generator()
config_path = generator.generate_and_save_config("notebook_test.yaml", "notebook_test")
print(f"✅ Created: {config_path}")

# Uncomment the following lines to run a new experiment:
# print("\n🚀 Running new experiment...")
# result = run_experiment_sync("notebook_test")
# if result["success"]:
#     print(f"✅ Experiment completed: {result['experiment_id']}")
#     print(f"   Consensus: {result['consensus_reached']}")
#     print(f"   Duration: {result['duration_seconds']:.1f}s")
# else:
#     print(f"❌ Experiment failed: {result['error']}")

print("\n💡 To run experiments, uncomment the code above or use the command line:")
print("   python run_experiment.py notebook_test")
print("   python run_batch.py config1 config2 --max-concurrent 2")

## Summary

This consolidated notebook provides:

1. **Data Loading**: Load experiment results from simplified JSON files
2. **Processing**: Extract key metrics and summaries
3. **Statistics**: Calculate basic statistics and performance metrics
4. **Visualization**: Create comprehensive analysis dashboard
5. **Export**: Save results for further analysis
6. **Integration**: Interface with simplified experiment system

**Usage:**
- Run cells sequentially to analyze existing experiments
- Modify `experiment_ids` list to analyze different experiments
- Use the simplified Python files for new experiments
- Export results for external analysis tools

**Next Steps:**
- Run more experiments using `run_experiment.py` and `run_batch.py`
- Add more sophisticated statistical analysis as needed
- Create custom visualizations for specific research questions